In [1]:
import pandas as pd
import numpy as np

In [2]:
countries = ['australia', 'canada', 'denmark', 'france', 'germany', 'italy', 'norway', 'spain', 'united-kingdom']

In [3]:
df_q = pd.read_excel('codebook.xlsx')
df_q.head(70)

,RecordNo,Unnamed: 1,Unnamed: 2
0,NaN,NaN,Value
1,Standard Attributes,Label,RecordNo
2,NaN,Type,Numeric
3,NaN,Measurement,Nominal
4,NaN,NaN,NaN
...,...,...,...
65,NaN,50,week 50
66,NaN,51,week 51
67,NaN,NaN,NaN
68,i1_health,NaN,NaN


In [4]:
df_q.drop(df_q.index[:67], inplace=True)

In [54]:
df_q

,RecordNo,Unnamed: 1,Unnamed: 2
67,NaN,NaN,NaN
68,i1_health,NaN,NaN
69,NaN,NaN,Value
70,Standard Attributes,Label,About how many people from your household have...
71,NaN,Type,Numeric
...,...,...,...
4632,NaN,WAH7_4,Less expensive living costs
4633,NaN,WAH7_5,To improve my / family health
4634,NaN,WAH7_6,It is better not to be in an urban place if th...
4635,NaN,WAH7_7,For environmental reasons


In [5]:
df_q.dropna(axis=0, subset=['RecordNo'], inplace=True) 

In [56]:
df_q.head(50)

,RecordNo,Unnamed: 1,Unnamed: 2
68,i1_health,NaN,NaN
70,Standard Attributes,Label,About how many people from your household have...
74,i2_health,NaN,NaN
76,Standard Attributes,Label,"Not including those people in your household, ..."
80,i7a_health,NaN,NaN
82,Standard Attributes,Label,How many times did you leave your home yesterd...
86,i3_health,NaN,NaN
88,Standard Attributes,Label,"In the last 7 days, have you personally been t..."
91,Valid Values,1,"Yes, and I tested positive"
96,i4_health,NaN,NaN


In [6]:
df_q.reset_index(drop=True, inplace=True)

In [7]:
for index, row in df_q.iterrows():
    if pd.isna(row['Unnamed: 2']) == True:
        df_q.iloc[index]['Unnamed: 2'] = df_q.iloc[index+1]['Unnamed: 2']
df_q.head(50)

,RecordNo,Unnamed: 1,Unnamed: 2
0,i1_health,NaN,About how many people from your household have...
1,Standard Attributes,Label,About how many people from your household have...
2,i2_health,NaN,"Not including those people in your household, ..."
3,Standard Attributes,Label,"Not including those people in your household, ..."
4,i7a_health,NaN,How many times did you leave your home yesterd...
5,Standard Attributes,Label,How many times did you leave your home yesterd...
6,i3_health,NaN,"In the last 7 days, have you personally been t..."
7,Standard Attributes,Label,"In the last 7 days, have you personally been t..."
8,Valid Values,1,"Yes, and I tested positive"
9,i4_health,NaN,"In the last 7 days, has someone in your househ..."


In [8]:
df_q=df_q[df_q['RecordNo'] != 'Standard Attributes']

In [9]:
df_q=df_q[df_q['RecordNo'] != 'Valid Values']

In [10]:
df_q.drop('Unnamed: 1', axis=1, inplace=True)

In [11]:
df_q.reset_index(drop=True, inplace=True)

In [12]:
df_q.head(50)

,RecordNo,Unnamed: 2
0,i1_health,About how many people from your household have...
1,i2_health,"Not including those people in your household, ..."
2,i7a_health,How many times did you leave your home yesterd...
3,i3_health,"In the last 7 days, have you personally been t..."
4,i4_health,"In the last 7 days, has someone in your househ..."
5,i5_health_1,Dry cough
6,i5_health_2,Fever
7,i5_health_3,Loss of sense of smell
8,i5_health_4,Loss of sense of taste
9,i5_health_5,Shortness of breath or difficulty breathing


In [13]:
df_q.rename(columns = {'RecordNo':'code', 'Unnamed: 2': 'question'}, inplace = True)
df_q

,code,question
0,i1_health,About how many people from your household have...
1,i2_health,"Not including those people in your household, ..."
2,i7a_health,How many times did you leave your home yesterd...
3,i3_health,"In the last 7 days, have you personally been t..."
4,i4_health,"In the last 7 days, has someone in your househ..."
...,...,...
505,Multiple Response Categories,I'm not eligible to receive one yet
506,$child_age,"In which, if any, any of the following age gro..."
507,Multiple Response Categories,0 - 2
508,$WAH7,If you are likely to further away from the cen...


In [14]:
code_dict={}
for index, row in df_q.iterrows():
    code = row['code']
    q = row['question']
    code_dict[code]=q
code_dict

{'i1_health': 'About how many people from your household have you come into physical contact with (within 2 meters / 6 feet)?',
 'i2_health': 'Not including those people in your household, about how many people have you come into physical contact with (within 2 meters / 6 feet)?',
 'i7a_health': 'How many times did you leave your home yesterday?  If you are not staying at home, how many times did you leave where you are staying yesterday?',
 'i3_health': 'In the last 7 days, have you personally been tested for coronavirus (COVID-19?)',
 'i4_health': 'In the last 7 days, has someone in your household been tested for coronavirus (COVID-19)?',
 'i5_health_1': 'Dry cough',
 'i5_health_2': 'Fever',
 'i5_health_3': 'Loss of sense of smell',
 'i5_health_4': 'Loss of sense of taste',
 'i5_health_5': 'Shortness of breath or difficulty breathing',
 'i5_health_99': 'None of these',
 'i5a_health': 'Thinking about the 7 days before your symptoms started, had you been in physical contact (within 2 m

In [113]:
codes = ['qweek', 
 'age', 'weight', 'gender', 'household_size', 'household_children','employment_status',
 'i1_health', 'i2_health', 'i7a_health', 'i3_health',  
 'i5_health_1', 'i5_health_2', 'i5_health_3', 'i5_health_4', 'i5_health_5', 'i5_health_99',
 'i5a_health', 'i6_health', 'i9_health', 'i10_health', 'i11_health', 'i12_health_1', 'i12_health_2',
 'i12_health_6','i12_health_8','i12_health_9','i12_health_10','i12_health_11', 'i12_health_12',
 'i12_health_13','i12_health_17','i12_health_18', 'i12_health_21',
 'm8_1', 'm8_2', 'm8_3', 'm8_4', 'm8_5', 'm8_6', 'm8_7', 'm8_8', 'm8_96', 'm8_99', 
 'd1_health_1', 'd1_health_2', 'd1_health_3','d1_health_4', 'd1_health_5', 'd1_health_6', 'd1_health_7', 'd1_health_8', 'd1_health_9',
 'd1_health_10', 'd1_health_11', 'd1_health_12', 'd1_health_13', 'd1_health_98', 'd1_health_99',
 'WCRex1', 'WCRex2', 'WCRV_4', 
 'Soc1_1', 'Soc1_2', 'Soc1_3', 
 'm1_1', 'm1_2', 'm1_3', 'm2', 'm3', 'm7_1', 'm7_2', 'm7_3', 'm7_4', 'm7_5', 'm7_6', 'm7_8', 'm7_9', 'm7_10', 'm7_11', 
 'm14_1', 'm14_2', 'm14_3', 'm14_4', 'm14_5', 'm14_6', 'm14_7', 'm14_8', 'm14_9', 'm14_10', 'm14_11', 'm14_96', 'm14_99',
 'v1', 'v2_1', 'v2_2','v2_3','v2_4','v2_5','v2_99', 'v3', 'v3_open', 'vac', 
 'vac_2', 'vac2_1', 'vac2_2', 'vac2_3', 'vac2_4', 'vac2_5', 'vac2_6', 'vac2_7', 'vac4', 'vac5', 'vac7','vac12_12',  
 'vac12_13', 'r1_8',
 'av1_1', 'av1_2', 'av1_3', 'av2',
 'r1_1', 'r1_2', 'r1_3', 'r1_4', 'r1_5', 'r1_6', 'r1_7',
 'ct2', 'ct4', 
 'ox1_1', 'ox1_2', 'ox3_1', 'ox3_2', 'ox3_4', 'ox6', 
 'w1', 'w2', 'w4_1', 'w4_2', 'w4_3', 'w4_6', 'w4_7', 'w4_8', 
 'work1', 'work2', 'work3', 'work4', 'work5',
 'cantril_ladder', 'PHQ4_1', 'PHQ4_2', 'PHQ4_3', 'PHQ4_4', 'w9_1', 'w9_2', 'w9_3', 'w9_4', 'w9_5']

In [16]:
emp_dict={
'employment_status_1': 'Full time employment',
'employment_status_2': 'Part time employment',
'employment_status_3': 'Full time student',
'employment_status_4': 'Retired',
'employment_status_5': 'Unemployed',
'employment_status_6': 'Not working',
'employment_status_7': 'Other'
}

In [17]:
def emp_helper(df):
    for index, row in df.iterrows():
        for k in emp_dict:
            if row[k] == 'Yes':
                df.loc[index, 'employment_status'] = emp_dict[k]
    drop_list=list(emp_dict.keys())
    df.drop(drop_list, axis=1, inplace=True)
    return df

In [31]:
dataframe_country_dict = {}
for country in countries:
    file_name=f'data/{country}.csv'
    print(country)
    df = pd.read_csv(file_name, low_memory=False)
    if country == 'denmark' or country == 'norway':
        df["employment_status"] = ""
        df = emp_helper(df)
    df=df[codes]
    df['Country'] = country
    df.rename(columns = code_dict, inplace = True)
    dataframe_country_dict[country] = df


australia
canada
denmark
france
germany
italy
norway
spain
united-kingdom


In [102]:
file_name=f'data/denmark.csv'
df_d = pd.read_csv(file_name, low_memory=False)
df_d["employment_status"] = ""
df_d = emp_helper(df_d)
df_d=df_d[codes]
df_d['Country'] = 'denmark'
df_d.rename(columns = code_dict, inplace = True)
dataframe_country_dict['denmark'] = df_d

In [70]:
df_a = dataframe_country_dict['denmark'] 
#df_a = df_a[df_a.qweek=='week 33']
df_a

,qweek,What is your age?,Weight,Gender,Number of people in household,Number of children under 18 in household,Employment Status,About how many people from your household have you come into physical contact with (within 2 meters / 6 feet)?,"Not including those people in your household, about how many people have you come into physical contact with (within 2 meters / 6 feet)?","How many times did you leave your home yesterday? If you are not staying at home, how many times did you leave where you are staying yesterday?",...,Little interest or pleasure in doing things,"Feeling down, depressed, or hopeless","Feeling nervous, anxious, or on edge",Not being able to stop or control worrying,I have felt cheerful and in good spirits,I have felt calm and relaxed,I have felt active and vigorous,I woke up feeling fresh and rested,My daily life has been filled with things that interest me,Country
0,week 1,25,2.150581,Male,1,0,Full time student,0,3,1,...,,,,,,,,,,denmark
1,week 1,42,0.857352,Male,2,0,Full time employment,3,3,4,...,,,,,,,,,,denmark
2,week 1,63,0.825933,Male,2,0,Retired,2,2,3,...,,,,,,,,,,denmark
3,week 1,30,0.889607,Female,2,0,Unemployed,1,0,1,...,,,,,,,,,,denmark
4,week 1,30,0.899496,Male,3,2,Full time employment,5,5,5,...,,,,,,,,,,denmark
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38147,week 38,47,1.427989,Female,2,0,Full time employment,,5,,...,Prefer not to say,Prefer not to say,Prefer not to say,Prefer not to say,,,,,,denmark
38148,week 38,49,1.493741,Male,4,1,Full time employment,,10,,...,Several days,Several days,More than half the days,More than half the days,,,,,,denmark
38149,week 38,68,1.023012,Female,2,0,Retired,,2,,...,Several days,Several days,More than half the days,Several days,,,,,,denmark
38150,week 38,19,0.858734,Female,1,0,Full time student,,15,,...,Nearly every day,Nearly every day,More than half the days,More than half the days,,,,,,denmark


In [63]:
dataframe_country_dict['denmark']['qweek'] = dataframe_country_dict['denmark']['qweek'].apply(lambda x: x[5:])
dataframe_country_dict['denmark']

,qweek,What is your age?,Weight,Gender,Number of people in household,Number of children under 18 in household,Employment Status,About how many people from your household have you come into physical contact with (within 2 meters / 6 feet)?,"Not including those people in your household, about how many people have you come into physical contact with (within 2 meters / 6 feet)?","How many times did you leave your home yesterday? If you are not staying at home, how many times did you leave where you are staying yesterday?",...,Little interest or pleasure in doing things,"Feeling down, depressed, or hopeless","Feeling nervous, anxious, or on edge",Not being able to stop or control worrying,I have felt cheerful and in good spirits,I have felt calm and relaxed,I have felt active and vigorous,I woke up feeling fresh and rested,My daily life has been filled with things that interest me,Country
0,1,25,2.150581,Male,1,0,Full time student,0,3,1,...,,,,,,,,,,denmark
1,1,42,0.857352,Male,2,0,Full time employment,3,3,4,...,,,,,,,,,,denmark
2,1,63,0.825933,Male,2,0,Retired,2,2,3,...,,,,,,,,,,denmark
3,1,30,0.889607,Female,2,0,Unemployed,1,0,1,...,,,,,,,,,,denmark
4,1,30,0.899496,Male,3,2,Full time employment,5,5,5,...,,,,,,,,,,denmark
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38147,38,47,1.427989,Female,2,0,Full time employment,,5,,...,Prefer not to say,Prefer not to say,Prefer not to say,Prefer not to say,,,,,,denmark
38148,38,49,1.493741,Male,4,1,Full time employment,,10,,...,Several days,Several days,More than half the days,More than half the days,,,,,,denmark
38149,38,68,1.023012,Female,2,0,Retired,,2,,...,Several days,Several days,More than half the days,Several days,,,,,,denmark
38150,38,19,0.858734,Female,1,0,Full time student,,15,,...,Nearly every day,Nearly every day,More than half the days,More than half the days,,,,,,denmark


In [64]:
dataframe_country_dict['denmark']['Gender'] = dataframe_country_dict['denmark']['Gender'].map( {'Male':0, 'Female':1})
dataframe_country_dict['denmark']['Gender'].dtype

dtype('int64')

In [86]:
households =['Number of people in household', 'Number of children under 18 in household']

In [90]:
for col in households:
    dataframe_country_dict['denmark'][col] = dataframe_country_dict['denmark'][col].apply(lambda x: x[:1]).astype(int)

In [103]:
dataframe_country_dict['denmark'] = pd.concat([dataframe_country_dict['denmark'], pd.get_dummies(dataframe_country_dict['denmark']['Employment Status'], drop_first=True, prefix='emp')], axis=1)
dataframe_country_dict['denmark'].drop('Employment Status', axis=1, inplace=True)

In [122]:
dataframe_country_dict['denmark']['If you were advised to do so by a healthcare professional or public health authority, how easy or difficult would it be for you be to self-isolate for 7 days?'] = dataframe_country_dict['denmark']['If you were advised to do so by a healthcare professional or public health authority, how easy or difficult would it be for you be to self-isolate for 7 days?'].map( {'Very easy':0, 'Somewhat easy':1, 'Neither easy nor difficult': 2, 'Somewhat difficult': 3, 'Very difficult': 4, 'Not sure': 5})
dataframe_country_dict['denmark']['If you were advised to do so by a healthcare professional or public health authority, how easy or difficult would it be for you be to self-isolate for 7 days?'].value_counts()


1.0    10900
0.0     8873
2.0     5159
3.0     3837
4.0     1583
5.0      798
Name: If you were advised to do so by a healthcare professional or public health authority, how easy or difficult would it be for you be to self-isolate for 7 days?, dtype: int64

In [124]:
dataframe_country_dict['denmark']['If you were advised to do so by a healthcare professional or public health authority to what extent are you willing or not to self-isolate for 7 days?'] = dataframe_country_dict['denmark']['If you were advised to do so by a healthcare professional or public health authority to what extent are you willing or not to self-isolate for 7 days?'].map( {'Very willing':0, 'Somewhat willing':1, 'Neither willing nor unwilling': 2, 'Somewhat unwilling': 3, 'Very unwilling': 4, 'Not sure': 5})
dataframe_country_dict['denmark']['If you were advised to do so by a healthcare professional or public health authority to what extent are you willing or not to self-isolate for 7 days?'].value_counts()


0.0    19289
1.0    10595
2.0     4378
3.0     1240
5.0      989
4.0      657
Name: If you were advised to do so by a healthcare professional or public health authority to what extent are you willing or not to self-isolate for 7 days?, dtype: int64

In [115]:
get_dummies = ['Employment Status', 'In the last 7 days, have you personally been tested for coronavirus (COVID-19?)', 
              ]
nans=['How many times did you leave your home yesterday?  If you are not staying at home, how many times did you leave where you are staying yesterday?',
      'About how many people from your household have you come into physical contact with (within 2 meters / 6 feet)?',
      'Thinking about the next 7 days… would you isolate yourself after feeling unwell or having any of the following new symptoms: a dry cough, fever, loss of sense of smell, loss of sense of taste, shortness of breath or difficulty breathing?',
     'In the last 7 days, have you personally been tested for coronavirus (COVID-19?)',
     ]

In [123]:
code_dict['i11_health']

'If you were advised to do so by a healthcare professional or public health authority to what extent are you willing or not to self-isolate for 7 days?'

In [ ]:
['i1_health', - int - nans?
, 
'i7a_health', - int - nans, del>15

'i3_health', - pd.get dummies - nans

        1- Yes, and I tested positive

        2- Yes, and I tested negative

        3- Yes, and I have not received my results from the test yet

        4- No, I have not

 


'i9_health',

        1- Yes

        2- No

        3- Not sure

 /

'i10_health', -- check for nans/dtype

/     

'i11_health' -- check for nans/dtype

/
 
'i12_health_1', -- can be int

        1- Always

        2- Frequently

        3- Soemtimes

        4- Rarely

        5- Not at all

       

'i12_health_2', -- can be int

        1- Always

        2- Frequently

        3- Soemtimes

        4- Rarely

        5- Not at all

 

 

'i12_health_6', -- can be int

        1- Always

        2- Frequently

        3- Soemtimes

        4- Rarely

        5- Not at all

 

'i12_health_8', -- can be int

        1- Always

        2- Frequently

        3- Soemtimes

        4- Rarely

        5- Not at all

 

'i12_health_9', -- can be int

        1- Always

        2- Frequently

        3- Soemtimes

        4- Rarely

        5- Not at all

 

'i12_health_10' -- can be int

        1- Always

        2- Frequently

        3- Soemtimes

        4- Rarely

        5- Not at all

 

'i12_health_11', -- can be int

        1- Always

        2- Frequently

        3- Soemtimes

        4- Rarely

        5- Not at all

 

'i12_health_12', -- can be int

        1- Always

        2- Frequently

        3- Soemtimes

        4- Rarely

        5- Not at all

 

'i12_health_13', -- can be int

        1- Always

        2- Frequently

        3- Soemtimes

        4- Rarely

        5- Not at all

 

'i12_health_17', -- can be int

        1- Always

        2- Frequently

        3- Soemtimes

        4- Rarely

        5- Not at all

 

'i12_health_18', -- can be int

        1- Always

        2- Frequently

        3- Soemtimes

        4- Rarely

        5- Not at all

 

'i12_health_21', -- can be int

        1- Always

        2- Frequently

        3- Soemtimes

        4- Rarely

        5- Not at all

 

 

 

ALL INT YES NO…

'd1_health_1',

'd1_health_2',

'd1_health_3',

'd1_health_4',

'd1_health_5',

'd1_health_6',

'd1_health_7',

'd1_health_8',

'd1_health_9',

'd1_health_10',

'd1_health_11',

'd1_health_12',

'd1_health_13',

'd1_health_98', prefer not to say

'd1_health_99', none of these - GET RID

 

 

'WCRex1', - int

        1- Very well

        2- Somewhat well

        3- Somewhat badly

        4- Very badly

        5- Don't know

 

'WCRex2', - int

        1- A lot of confidence

        2- A fair amount of confidence

        3- Not very much confidence

        4- No confidence at all

        5- Don't know

 

'WCRV_4', - int

        1- I am very scared that I will contract the Coronavirus (COVID-19)

        2- I am fairly scared that I will contract the Coronavirus (COVID-19)

        3- I am not very scared that I will contract the Coronavirus (COVID-19)

        4- I am not at all scared that I will contract the Coronavirus (COVID-19)

        977- Don't know

        988- Not applicable -I have already contracted Coronavirus (COVID-19)

 

'Soc1_1', - ALL int Yes, No

'Soc1_2',

'Soc1_3',

 

 

'm1_1', -- can be int
        1- Very willing

        2- Somewhat willing

        3- Neither willing nor unwilling

        4- Somewhat unwilling

        5- Very unwilling

        99- Not sure

 

'm1_2', -- can be int
        1- Very willing

        2- Somewhat willing

        3- Neither willing nor unwilling

        4- Somewhat unwilling

        5- Very unwilling

        99- Not sure

 

'm1_3', -- can be int
        1- Very willing

        2- Somewhat willing

        3- Neither willing nor unwilling

        4- Somewhat unwilling

        5- Very unwilling

        99- Not sure

 

'm2', int

'm3', int

'm7_1', - int

        1- Very much

        2- Somewhat

        3- Not at all

        99- Not sure

 

'm7_2', - int

        1- Very much

        2- Somewhat

        3- Not at all

        99- Not sure

 

'm7_3', - int

        1- Very much

        2- Somewhat

        3- Not at all

        99- Not sure

 

'm7_4', - int

        1- Very much

        2- Somewhat

        3- Not at all

        99- Not sure

 

'm7_5', - int

        1- Very much

        2- Somewhat

        3- Not at all

        99- Not sure

 

'm7_6', - int

        1- Very much

        2- Somewhat

        3- Not at all

        99- Not sure

 

'm7_8', - int

        1- Very much

        2- Somewhat

        3- Not at all

        99- Not sure

 

'm7_9', - int

        1- Very much

        2- Somewhat

        3- Not at all

        99- Not sure

 

'm7_10', - int

        1- Very much

        2- Somewhat

        3- Not at all

        99- Not sure

 

'm7_11', - int

        1- Very much

        2- Somewhat

        3- Not at all

        99- Not sure

 

'm14_1', - int

        0- No

        1- Yes

'm14_2'- int

        0- No

        1- Yes

,

'm14_3'- int

        0- No

        1- Yes

,

'm14_4'- int

        0- No

        1- Yes

,

'm14_5'- int

        0- No

        1- Yes

,

'm14_6'- int

        0- No

        1- Yes

,

'm14_7'- int

        0- No

        1- Yes

,

'm14_8'- int

        0- No

        1- Yes

,

'm14_9'- int

        0- No

        1- Yes

,

'm14_10'- int

        0- No

        1- Yes

,

'm14_11'- int

        0- No

        1- Yes

,

'm14_96'- int

        0- No

        1- Yes

,

'm14_99'- int

        0- No

        1- Yes

,

 

 

 

'v1', int-

        1- Yes

        2- No

        3- Not sure

 

'v2_1', int-

        0- No

        1- Yes

'v2_2', int-

        0- No

        1- Yes

 

'v2_3', int-

        0- No

        1- Yes

 

'v2_4', int-

        0- No

        1- Yes

 

'v2_5', int-

        0- No

        1- Yes

 

'v2_99', int-

        0- No

        1- Yes


'v3', - dummies

      1- The government said that people can leave home only for essential services

      2- A healthcare provider recommended delaying or missing vaccines

      3- The vaccination clinic is closed or not giving vaccines due to COVID-19

      4- It's hard to get an appointment even though the vaccination clinic is open

      5- Couldn't afford it

      6- Wanted to save health services for the people who really need them now

      7- People don't need vaccines now because no one is going out

      8- Worry about getting COVID-19 by leaving leave the house

      9- Worry about getting COVID-19 when on public transit to get to the vaccination clinic

      10- Worry about getting COVID-19 at the vaccination clinic

      11- Worry about giving COVID-19 to other people at the vaccination clinic

                99- Something else

 

'vac', - int

        1- No, neither

        2- Yes, one dose

        3- Yes, two doses


'vac4', -int

        1- Not at all important

        2- A little important

        3- Moderately important

        4- Very important

'vac6', -int

        0- No

        1- Yes

        99- Not sure

'vac7', -int

        1- Not at all

        2- A little

        3- Moderately

        4- Very much

'vac12_12',  ALL in No, Yes

'vac12_13',

 


'av1_1', ALL int No, Yes

'av1_2',

'av1_3',

'av2', -int

        1- Yes, I think more research needs to be done before I will consider receiving the vaccine

        2- No, I don't think more research needs to be done before I will consider receiving the vaccine

        99- Not sure

 

'r1_1' - int, 1-7, Disagre, Agree

'r1_2'- int,

'r1_3'- int,

'r1_4'- int,

'r1_5'- int,

'r1_6'- int,

'r1_7'- int,

'r1_8'- int

'r1_9'- int

 

 

'ct2', - int

        0- No

        1- Yes

        99- Not sure

 

'ct4', - int

        0- No

        1- Yes

        99- Not sure

 

 


'ox1_1', - int

        1- Much lower

        2-

        3- The same

        4-

        5- Much higher

 

'ox1_2', - int

        1- Much lower

        2-

        3- The same

        4-

        5- Much higher

 

'ox3_1', - int

        1- Strongly agree

        5- Strongly disagree

'ox3_2', - int

        1- Strongly agree

        5- Strongly disagree

 

'ox3_4', - int

        1- Strongly agree

        5- Strongly disagree

 

'ox6', - int

        1- Not efficient at all

        2- Extremely efficient

 

'w1', - int

        1- More united

        2- No change

        3- More divided

 

'w2', - int

        1- Very stong

        2- Somewhat strong

        3- Somewhat weak

        4- Very weak

 

'w4_1', -- ALL int No, Yes

'w4_2',

'w4_3',

'w4_6',

'w4_7',

'w4_8',

 

'WAH4' - int, YB, NS

 

'work1', - int

        1- Yes, fully

        2- Yes, partly

        3- No, not at all

'work2', -int

        1- No

        2- Yes, within past 2 weeks

        3- Yes, between 2 weeks and 2 months ago

        4- Yes, more than two months ago but since 1st February 2020


'work3', -int

        1- No

        2- Yes, that started within past 2 weeks

        3- Yes, between 2 weeks and 2 months ago

        4- Yes, more than two months ago but since 1st February 2020

 

'work4', -int

        1- Very likely

        2- Quite likely

        3- Neother likely nor unlikely

        4- Quite unlikely

        5- Very unlikely

        98- Don't know

 

'work5', - int

        1- Yes, fully

        2- Yes, partly

        3- No, not at all

 

 

'cantril_ladder', -- INT, 0-10

'PHQ4_1', - int

        1- Not at all

        2- Several days

        3- More than half the days

        4- Nearly every day

        99- Prefer not to say

 

'PHQ4_2', - int

        1- Not at all

        2- Several days

        3- More than half the days

        4- Nearly every day

        99- Prefer not to say

 

'PHQ4_3', - int

        1- Not at all

        2- Several days

        3- More than half the days

        4- Nearly every day

        99- Prefer not to say

 

'PHQ4_4', - int

        1- Not at all

        2- Several days

        3- More than half the days

        4- Nearly every day

        99- Prefer not to say

 

'w9_1', - int

        1- At no time

        2- Some of the time

        3- Less than half of the time

        4- More than half of the time

        5- Most of the time

        6- All the time

'w9_2', - int

        1- At no time

        2- Some of the time

        3- Less than half of the time

        4- More than half of the time

        5- Most of the time

        6- All the time

 

'w9_3', - int

        1- At no time

        2- Some of the time

        3- Less than half of the time

        4- More than half of the time

        5- Most of the time

        6- All the time

 

'w9_4', - int

        1- At no time

        2- Some of the time

        3- Less than half of the time

        4- More than half of the time

        5- Most of the time

        6- All the time

 

'w9_5'- int

        1- At no time

        2- Some of the time

        3- Less than half of the time

        4- More than half of the time

        5- Most of the time

        6- All the time

]